In [935]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.optimize import minimize
from scipy.stats import gaussian_kde

from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import DistanceMetric

import networkx as nx

import matplotlib.pyplot as plt

In [976]:
data = pd.read_excel('ENB2012_data.xlsx')
del data['Y2']
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84


In [977]:
data.shape

(768, 9)

In [978]:
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000


In [979]:
X = data[['X1','X2','X3','X4','X5','X6','X7','X8']]
y = data[['Y1']]

scaler_X = MinMaxScaler().fit(X)
X_scaled = pd.DataFrame(scaler_X.transform(X), columns=['X1','X2','X3','X4','X5','X6','X7','X8'])

scaler_y = MinMaxScaler().fit(y)
y_scaled = pd.DataFrame(scaler_y.transform(y), columns=['Y1'])

In [980]:
regr = linear_model.LinearRegression()
svm = SVR(gamma='scale', C=2, epsilon=0.1)
ran_for = RandomForestRegressor(n_estimators=100)

In [981]:
model = [regr, svm, ran_for]
for mod in model:
    print(mod)
    scores = cross_val_score(mod, X_scaled, y_scaled.values.ravel(), scoring='neg_mean_absolute_error', cv=5)
    print('Mean score: ', -scores.mean())
    print('SD:', scores.std())
    print('\n')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Mean score:  0.06224443840639384
SD: 0.009799382829484463


SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Mean score:  0.07959787502500451
SD: 0.03290389924038814


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
Mean score:  0.021175149797012953
SD: 0.01850601174652489




In [982]:
ran_for.fit(X_scaled, y_scaled.values.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [983]:
for col in list(data)[:8]:
    print(f'{col} values: ',[np.sort(data[col].unique())])

X1 values:  [array([0.62, 0.64, 0.66, 0.69, 0.71, 0.74, 0.76, 0.79, 0.82, 0.86, 0.9 ,
       0.98])]
X2 values:  [array([514.5, 563.5, 588. , 612.5, 637. , 661.5, 686. , 710.5, 735. ,
       759.5, 784. , 808.5])]
X3 values:  [array([245. , 269.5, 294. , 318.5, 343. , 367.5, 416.5])]
X4 values:  [array([110.25, 122.5 , 147.  , 220.5 ])]
X5 values:  [array([3.5, 7. ])]
X6 values:  [array([2, 3, 4, 5])]
X7 values:  [array([0.  , 0.1 , 0.25, 0.4 ])]
X8 values:  [array([0, 1, 2, 3, 4, 5])]


In [984]:
dist = DistanceMetric.get_metric('manhattan')
np.set_printoptions(suppress=True)

In [1057]:
example = np.array([0.82, 637, 367.5, 122.5, 7, 4, 0.25, 4]).reshape(1, -1)
eg_scaled = scaler_X.transform(example)
eg_score = ran_for.predict(eg_scaled)
eg_HL = scaler_y.inverse_transform(eg_score.reshape(1, -1))
eg_HL

array([[34.44]])

In [1064]:
HL_target = 0.80*eg_HL
HL_target

array([[27.552]])

In [1065]:
# nearest neighbour
counter_data = data[data['Y1'] <= HL_target[0][0]]
counter_data.reset_index(drop=True, inplace=True)
distances = dist.pairwise(counter_data[['X1','X2','X3','X4','X5','X6','X7','X8']], example)
min_dist_index = np.argmin(distances)
cf_nn = counter_data.loc[min_dist_index][:8].values
cf_nn_HL = counter_data.loc[min_dist_index][-1]
print('Nearest neighbour ',cf_nn)
print('cf HL',cf_nn_HL)

Nearest neighbour  [  0.76 661.5  416.5  122.5    7.     4.     0.     0.  ]
cf HL 24.77


In [1031]:
def dist_mad(cf, eg):
    manhat = [cdist(eg.T, cf.reshape(1,-1).T ,metric='cityblock')[i][i] for i in range(len(eg.T))]
    mad = stats.median_absolute_deviation(X)
    return sum(manhat/mad)

def loss_function_mad(x_dash):
    L = lamda*ran_for.predict(x_dash.reshape(1,-1)) + dist_mad(x_dash.reshape(1,-1), eg_scaled)
    return L

In [1066]:
# initial conditions
lamda = 0.1 
x0 = data.mean()[:-1].values.reshape(1,-1) # initial guess for cf
x0 = x0 + np.random.normal(0,0.1,8) * x0

bnds = ((0, 1),(0, 1),(0, 1),(0, 1),(0, 1),(0, 1),(0, 1),(0, 1))
res = minimize(loss_function_mad, x0, method='SLSQP', bounds=bnds, options={'maxiter':1000})
cf = res.x.reshape(1, -1)

cf_score = ran_for.predict(cf)
cf_wach_HL = scaler_y.inverse_transform(cf_score.reshape(-1, 1))

i=0
while HL_target < cf_wach_HL:
    lamda += 0.025
    x0 = cf # starting point is current cf
    res = minimize(loss_function_mad, x0, method='SLSQP', bounds=bnds, options={'maxiter':1000})
    cf = res.x.reshape(1, -1)
    
    cf_score = ran_for.predict(cf)
    cf_wach_HL = scaler_y.inverse_transform(cf_score.reshape(-1, 1))
    
    i += 1
    if i == 5000:
        print('Error condition not met after',i,'iterations')
        break
        
cf_wach = scaler_X.inverse_transform(cf)
        
print('Iterations: ',i)
print('Counterfactual point: ',cf_wach)
print('with HL prediction of ',cf_wach_HL)

Iterations:  317
Counterfactual point:  [[  0.818 624.75  330.666 117.107   6.991   4.033   0.252   4.028]]
with HL prediction of  [[25.577]]


In [1033]:
def dist_face(x,y):
    return cdist(x.reshape(1,-1), y.reshape(1,-1), metric='cityblock')

# nearest neighbour
dist = DistanceMetric.get_metric('cityblock')
def nearest(example):
    distances = dist.pairwise(X_scaled, example)
    index_min = distances.argmin()
    nearest_data = X_scaled.iloc[index_min].values
    return nearest_data, index_min

In [1128]:
dist_threshold = 1.2
density_threshold = 1.5e+9

data_example, start_node = nearest(eg_scaled)
if dist_face(eg_scaled, data_example)[0][0] > dist_threshold:
    print('Data point does not meet distance threshold')

In [1129]:
# create weights based on distance threshold
kernel = gaussian_kde(X_scaled.values.T)

m, n = X_scaled.shape
w_ij = [0] * int(m*(m-1)*0.5)
edge_weights = []
k = 0
q = 0
for edge_from in range(m):
    k = 1 + edge_from
    for edge_to in range(k,m):
        if dist_face(X_scaled.iloc[edge_from].values, X_scaled.iloc[edge_to].values)[0][0] < dist_threshold:
            w_ij[q] = np.log(kernel((X_scaled.iloc[edge_from].values + X_scaled.iloc[edge_to].values)/2)[0] * \
                              dist_face(X_scaled.iloc[edge_from].values, X_scaled.iloc[edge_to].values)[0][0])
        edge_weights.append((edge_from, edge_to, {'weight':w_ij[q]}))
        q += 1

nonzero_edge = []
for i in range(len(edge_weights)):
    if edge_weights[i][2]['weight'] != 0:
        nonzero_edge.append(edge_weights[i])

In [1130]:
# create graph with nodes that meet density threshold
G = nx.Graph()

G.add_nodes_from(range(len(X_scaled)))
G.add_edges_from(nonzero_edge)

low_density = []
for i in range(m):
    if kernel(X_scaled.iloc[i].values) < density_threshold:
        low_density.append(i)
G.remove_nodes_from(low_density)

print('nodes that meet density theshold: ', G.number_of_nodes())
print('edges that meet distance theshold: ', G.number_of_edges())

nodes that meet density theshold:  672
edges that meet distance theshold:  26048


In [1131]:
target_nodes = list(G.nodes())
index = [i for i,node in enumerate(target_nodes) if node == start_node][0]
del target_nodes[index]

pred_threshold_fail = []
for i in range(len(X_scaled)):
    face_score = ran_for.predict(X_scaled.iloc[i].values.reshape(1,-1))
    face_HL = scaler_y.inverse_transform(face_score.reshape(1, -1))
    if face_HL > HL_target:
        pred_threshold_fail.append(i)

length, path = nx.multi_source_dijkstra(G,target_nodes,target=start_node)
path = path[::-1]

while path[-1] in pred_threshold_fail:
    index = [i for i,node in enumerate(target_nodes) if node == path[-1]][0]
    del target_nodes[index]
          
    length, path = nx.multi_source_dijkstra(G,target_nodes,target=start_node)
    path = path[::-1]

In [1132]:
for i, node in enumerate(path[1:]):
    point = scaler_X.inverse_transform(X_scaled.iloc[node].values.reshape(1, -1))
    print(f'Path point {i+1}: ',[(f'X{j+1}: {round(point[0][j],2)}') for j in range(8)])

    face_score = ran_for.predict(X_scaled.iloc[path[-1]].values.reshape(1,-1))
face_HL = scaler_y.inverse_transform(face_score.reshape(1, -1))
print('Final point HL:',round(face_HL[0][0],1))

Path point 1:  ['X1: 0.82', 'X2: 612.5', 'X3: 318.5', 'X4: 147.0', 'X5: 7.0', 'X6: 4.0', 'X7: 0.25', 'X8: 4.0']
Final point HL: 25.4


In [1099]:
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000


In [1100]:
print('Example: ',[(f'X{i+1}: {example[0][i]}') for i in range(8)])
print('Example HL:',round(eg_HL[0][0],1))
print('Target HL:',round(HL_target[0][0],1))

Example:  ['X1: 0.82', 'X2: 637.0', 'X3: 367.5', 'X4: 122.5', 'X5: 7.0', 'X6: 4.0', 'X7: 0.25', 'X8: 4.0']
Example HL: 34.4
Target HL: 27.6


In [1101]:
print('Nearest neighbour cf ',[(f'X{i+1}: {cf_nn[i]}') for i in range(8)])
print('CF HL:',cf_nn_HL)

Nearest neighbour cf  ['X1: 0.76', 'X2: 661.5', 'X3: 416.5', 'X4: 122.5', 'X5: 7.0', 'X6: 4.0', 'X7: 0.0', 'X8: 0.0']
CF HL: 24.77


In [1102]:
print('Wachter cf: ',[(f'X{i+1}: {round(cf_wach[0][i],1)}') for i in range(8)])
print('CF HL:',round(cf_wach_HL[0][0],1))

Wachter cf:  ['X1: 0.8', 'X2: 624.7', 'X3: 330.7', 'X4: 117.1', 'X5: 7.0', 'X6: 4.0', 'X7: 0.3', 'X8: 4.0']
CF HL: 25.6


In [1103]:
for i, node in enumerate(path[1:]):
    point = scaler_X.inverse_transform(X_scaled.iloc[node].values.reshape(1, -1))
    print(f'Path point {i+1}: ',[(f'X{j+1}: {round(point[0][j],1)}') for j in range(8)])
face_score = ran_for.predict(X_scaled.iloc[path[-1]].values.reshape(1,-1))
face_HL = scaler_y.inverse_transform(face_score.reshape(1, -1))
print('Final point HL:',round(face_HL[0][0],1))

Path point 1:  ['X1: 0.8', 'X2: 612.5', 'X3: 318.5', 'X4: 147.0', 'X5: 7.0', 'X6: 4.0', 'X7: 0.2', 'X8: 4.0']
Final point HL: 25.4
